# X-ray photoelectron spectroscopy (XPS)
For the source code, see [xps](https://workgraph-collections.readthedocs.io/en/latest/ase/espresso/module.html#workgraph_collections.ase.espresso.xps.xps_workgraph).


## Example: ETFA molecule

### Prepare the inputs and submit the workflow



In [ ]:
from ase.io import read
from aiida import load_profile
from workgraph_collections.ase.espresso.xps import XpsWorkgraph

load_profile()

# create input structure
atoms = read("../datas/ETFA.xyz")
atoms.center(vacuum=5.0)
input_data = {
        "CONTROL": {
            "calculation": "scf",
        },
        "SYSTEM": {
            "ecutwfc": 50,
            "ecutrho": 400,
            "occupations": "fixed",
        },
    }
kpts = (1, 1, 1)
# Pseudos from https://github.com/superstar54/xps-data/tree/main/pseudo_demo/pseudo_demo_pbe
pseudopotentials = {
    "H": "H.pbe-kjpaw_psl.1.0.0.UPF",
    "F": "F.pbe-n-rrkjus_psl.1.0.0.UPF",
}
# corrections from https://github.com/superstar54/xps-data/blob/main/pseudo_demo/pseudo_demo_pbe/datas.py
core_hole_pseudos = {
    "C": {"ground": "C.pbe-n-kjgipaw_psl.1.0.0.UPF",
          "core_hole": "C.star1s.pbe-n-kjgipaw_psl.1.0.0.UPF",
          "correction": 345.99 - 6.2,
    },
    "O": {"ground": "O.pbe-n-kjpaw_psl.0.1.UPF",
          "core_hole": "O.star1s.pbe-n-kjpaw_psl.0.1.UPF",
          "correction": 676.47 - 8.25,
    }
}
#
metadata = {
    "options": {
        'prepend_text' : """eval "$(conda shell.posix hook)"
        conda activate aiida
        export OMP_NUM_THREADS=1
        """,
    }
}
scf_inputs = {
    "command": "mpirun -np 4 pw.x",
    "computer": "localhost",
    "metadata": metadata,
    "input_data": input_data,
    "kpts": kpts,
    "pseudopotentials": pseudopotentials,
    "pseudo_dir": "/home/wang_x3/datas/pseudos/xps/pbe",
    "core_hole_treatment": "xch",
}

# ===============================================================================
wg = XpsWorkgraph.build(
    atoms=atoms,
    marked_structures_inputs={"atom_list": [0, 1, 2, 3],
                              "min_cell_length": 14,
                              "is_molecule": True,
                              },
    scf_inputs=scf_inputs,
    core_hole_pseudos=core_hole_pseudos,
    metadata=metadata,
)
wg

WorkGraph process created, PK: 35068


<WorkChainNode: uuid: 71341155-996d-4cb3-85a8-9d0caa328b1a (pk: 35068) (aiida.workflows:workgraph.engine)>

Run the workflow

In [ ]:
wg.run()

## Print the results


In [5]:
for key, energy in wg.tasks["get_binding_energy"].outputs["result"].value.value.items():
    print(f"{key}: {energy:.2f} eV")

C_0: 298.32 eV
C_1: 295.27 eV
C_2: 293.01 eV
C_3: 291.45 eV


### Run on HPC
Here is an example of how to run the pw.x calculation on the CSCS eiger cluster.


```python 
# run on CSCS eiger cluster
metadata_eiger = {
    "options": {
        'prepend_text' : """
module load cray/22.05  cpeIntel/22.05 QuantumESPRESSO/7.0
eval "$(/users/xingwang/miniconda3/bin/conda shell.posix hook)"
conda activate py3.11
export OMP_NUM_THREADS=1
        """,
        'custom_scheduler_commands' : '#SBATCH --account=mr32',
        'resources': {
                    'num_machines' : 1,
                    'num_mpiprocs_per_machine' : 128,
        }
    }
}
scf_inputs = {
    "command": "srun -n 128 pw.x",
    "computer": "eiger",
    "metadata": metadata_eiger,
    "input_data": input_data,
    "kpts": kpts,
    "pseudopotentials": pseudopotentials,
    # "pseudo_dir": "/home/wang_x3/datas/pseudos/xps/pbe",
    "pseudo_dir": "/users/xingwang/datas/pseudos/xps/pbe",
    "core_hole_pseudos": core_hole_pseudos,
    "is_molecule": True,
}
```